# 데이터셋 준비하기
- https://www.kaggle.com/code/mitanshuchakrawarty/fine-tune-llm-for-text-summary

## 1. 환경 셋업

In [1]:
from dotenv import load_dotenv

import os

HF_TOKEN = os.getenv('HF_TOKEN')

!huggingface-cli login --token {HF_TOKEN}

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [2]:
import os 
os.environ['TRANSFORMERS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_DATASETS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_HOME'] = "/home/ec2-user/SageMaker/.cache"

In [3]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset, load_metric
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 2. 데이터 셋 준비
### 데이터 셋 다운로드

In [4]:
huggingface_dataset_name = "daekeun-ml/naver-news-summarization-ko"

# dataset = load_dataset(huggingface_dataset_name, "3.0.0")
dataset = load_dataset(huggingface_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [5]:
dataset["train"][0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

## 3. 데이터셋 변형

### Chat Message 형태 템플릿 정의

In [6]:
import json

def format_instruction(system_prompt: str, article: str, summary: str):
    message = [
            {
                'content': system_prompt,
                'role': 'system'
            },
            {
                'content': f'Please summarize the goals for journalist in this text:\n\n{article}',
                'role': 'user'
            },
            {
                'content': f'{summary}',
                'role': 'assistant'
            }
        ]
    
    return message # json.dumps(message, indent=2) # json.dumps(message, ensure_ascii=False, indent=2)


# 사용 예시
# system_prompt = "You are an AI assistant specialized in news articles. Your role is to provide accurate summaries and insights. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings."
# article = "Within three days, the intertwined cup nest of grasses was complete, featuring a canopy of overhanging grasses to conceal it. And decades later, it served as Rinkert's portal to the past inside the California Academy of Sciences. Information gleaned from such nests, woven long ago from species in plant communities called transitional habitat, could help restore the shoreline in the future. Transitional habitat has nearly disappeared from the San Francisco Bay, and scientists need a clearer picture of its original species composition—which was never properly documented. With that insight, conservation research groups like the San Francisco Bay Bird Observatory can help guide best practices when restoring the native habitat that has long served as critical refuge for imperiled birds and animals as adjacent marshes flood more with rising sea levels. \"We can't ask restoration ecologists to plant nonnative species or to just take their best guess and throw things out there,\" says Rinkert."
# summary = "Scientists are studying nests hoping to learn about transitional habitats that could help restore the shoreline of San Francisco Bay."

# print(format_instruction(system_prompt, article, summary))

### Chat Message 형태로 변환

In [7]:
# Add system message to each conversation
columns_to_remove = list(dataset["train"].features)
columns_to_remove

['date', 'category', 'press', 'title', 'document', 'link', 'summary']

In [8]:

def generate_instruction_dataset(data_point):
    system_prompt = "You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings."

    return {
        "messages": format_instruction(system_prompt, data_point["document"],data_point["summary"])
    }

def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset).remove_columns(columns_to_remove)
    )    

##### 전체 데이터 셋에서 일부 데티터 추출 (짧은 실습을 위해서)

In [9]:
train_num_debug_samples = 10
test_num_debug_samples = 10

In [10]:
## APPLYING PREPROCESSING ON WHOLE DATASET

dataset["train"] = process_dataset(dataset["train"].select(range(train_num_debug_samples)))
dataset["test"] = process_dataset(dataset["validation"])
dataset["validation"] = process_dataset(dataset["validation"])

In [11]:
# Select 1000 rows from the training split
train_dataset = dataset['train'].shuffle(seed=42).select([i for i in range(train_num_debug_samples)])

# Select 100 rows from the test and validation splits
test_dataset = dataset['test'].shuffle(seed=42).select([i for i in range(test_num_debug_samples)])
validation_dataset = dataset['validation'].shuffle(seed=42).select([i for i in range(test_num_debug_samples)])


In [12]:
train_dataset,test_dataset,validation_dataset

(Dataset({
     features: ['messages'],
     num_rows: 10
 }),
 Dataset({
     features: ['messages'],
     num_rows: 10
 }),
 Dataset({
     features: ['messages'],
     num_rows: 10
 }))

In [13]:
train_dataset[0]

{'messages': [{'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.',
   'role': 'system'},
  {'content': 'Please summarize the goals for journalist in this text:\n\n한탄바이러스 발견 노벨상 유력 후보로 자주 거론 한국을 대표하는 의학자이자 미생물학자 이호왕 고려대 명예교수. 고려대의대 제공 한탄바이러스를 발견한 우리나라 대표 의과학자 이호왕 고려대 명예교수가 5일 숙환으로 별세했다. 향년 94세. 고인은 바이러스의 병원체와 진단법 백신까지 모두 개발한 한국을 대표하는 의학자이자 미생물학자다. 신증후군출혈열 병원체인 한탄바이러스와 서울바이러스를 세계 최초로 발견하고 예방백신 및 진단법을 개발해 세계 의학발전에 기여한 것으로 평가된다. 1928년 함경남도 신흥에서 출생한 고인은 1973년 고대의대에 부임해 의과대학장을 지냈으며 1982년 세계보건기구 신증후출혈열연구협력센터 소장 2000년 대한민국학술원 회장 등을 역임했다. 1979년 미국 최고민간인공로훈장 1987년 인촌상 1992년 호암상 1995년 태국 프린스 마히돌상 2001년 일본 니케이 아시아상 2002년 과학기술훈장 창조장 2009년 서재필의학상 2018년 대한민국 과학기술유공자로 추대됐으며 2002년 미국 학술원 NAS 외국회원 2009년 일본 학사원 명예회원에 선정되며 국내외 학계에서 활발한 활동을 이어왔다. 고인의 빈소는 고려대 안암병원 장례식장 303호에 마련됐다. 발인은 7일 오전 11시 50분이고 장지는 서울추모공원이다

## 4. 데이터 셋을 JSON 으로 저장

In [14]:
import os

dataset_name = huggingface_dataset_name.split("/")[1]
data_folder = os.path.join("../data/",dataset_name)
os.makedirs(data_folder, exist_ok=True)

train_data_json = os.path.join(data_folder, "train_dataset.json")
validation_data_json = os.path.join(data_folder, "validation_dataset.json")
test_data_json = os.path.join(data_folder, "test_dataset.json")
print("train_data_json: ", train_data_json)
print("validation_data_json: ", validation_data_json)
print("test_data_json: ", test_data_json)

train_data_json:  ../data/naver-news-summarization-ko/train_dataset.json
validation_data_json:  ../data/naver-news-summarization-ko/validation_dataset.json
test_data_json:  ../data/naver-news-summarization-ko/test_dataset.json


In [15]:
# save datasets to disk 
train_dataset.to_json(train_data_json, orient="records", force_ascii=False)
validation_dataset.to_json(validation_data_json, orient="records", force_ascii=False)
test_dataset.to_json(test_data_json, orient="records", force_ascii=False)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

29075

### 다음 노트북에서 사용하기 위해 변수 저장

In [16]:
%store data_folder
%store train_data_json 
%store validation_data_json 
%store test_data_json 


Stored 'data_folder' (str)
Stored 'train_data_json' (str)
Stored 'validation_data_json' (str)
Stored 'test_data_json' (str)


## 5. Option: 데이터 셋을 ChatTemplate 형태로 바꾸기

### Chat Template 정의

In [17]:
LLAMA_3_CHAT_TEMPLATE = (
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{ message['content'] }}"
        "{% elif message['role'] == 'user' %}"
            "{{ '\n\nHuman: ' + message['content'] +  eos_token }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '\n\nAssistant: '  + message['content'] +  eos_token  }}"
        "{% endif %}"
    "{% endfor %}"
    "{% if add_generation_prompt %}"
    "{{ '\n\nAssistant: ' }}"
    "{% endif %}"
)
LLAMA_3_CHAT_TEMPLATE

"{% for message in messages %}{% if message['role'] == 'system' %}{{ message['content'] }}{% elif message['role'] == 'user' %}{{ '\n\nHuman: ' + message['content'] +  eos_token }}{% elif message['role'] == 'assistant' %}{{ '\n\nAssistant: '  + message['content'] +  eos_token  }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '\n\nAssistant: ' }}{% endif %}"

### Chat Template 으로 변형하기

In [18]:
# Tokenizer        
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE

# template dataset
def template_dataset(examples):
    return{"text":  tokenizer.apply_chat_template(examples["messages"], tokenize=False)}

train_dataset = train_dataset.map(template_dataset, remove_columns=["messages"])
test_dataset = test_dataset.map(template_dataset, remove_columns=["messages"])
validation_dataset = validation_dataset.map(template_dataset, remove_columns=["messages"])    

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### 변형된 Chat Message 형태 예시 보기

In [ ]:
# print random sample
import random

for index in random.sample(range(len(train_dataset)), 1):
    print("index: ", index)
    # index = 5343
    print(train_dataset[index]["text"])